# Introduction to Time Series Analysis

For a more detailed breakdown of the theory behind the code, check out the [full article](https://blog.paperspace.com/introduction-time-series-analysis) on the Paperspace blog.

In [ ]:
# Download the data
!wget https://s3.amazonaws.com/ps.public.resources/ml-showcase/jena_climate_2009_2016.csv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


df = pd.read_csv('jena_climate_2009_2016.csv')

time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')
series = df['T (degC)']
series.index = time

print(df)

In [ ]:
series.plot()
plt.ylabel('Temp')
plt.show()

In [ ]:
import numpy as np

plt.plot(series.index, np.array([series.mean()] * len(series)))

x = np.arange(len(series))
y = series.values
m, c = np.polyfit(x, y, 1)

plt.plot(series.index, m*x + c)

series.rolling(3600).mean().plot()

plt.legend(['mean', 'regression line', 'rolling mean'])
plt.ylabel('Temp')
plt.show()


roll_std = series.rolling(3600).std()
roll_std.dropna(inplace=True)
plt.plot(roll_std.index, np.array([roll_std.mean()] * len(roll_std)))

x = np.arange(len(roll_std))
y = roll_std.values
m, c = np.polyfit(x, y, 1)

plt.plot(roll_std.index, m*x + c)

roll_std.plot()
plt.legend(['rolling std mean', 'rolling std regression', 'rolling std'])
plt.ylabel('Temp')
plt.show()

In [ ]:
def correlation(x, y):
    x_norm = x - x.mean()
    y_norm = y - y.mean()
    return np.sum(x_norm * y_norm) / np.sqrt(np.sum(x_norm ** 2) * np.sum(y_norm ** 2))

In [ ]:
def autocorrelation(x, k):
    val_0 = np.sum((x - x.mean()) ** 2) / len(x)
    val_k = np.sum((x[:-k] - x.mean()) * (x[k:] - x.mean())) / len(x)
    return val_k / val_0

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def get_acf_pacf_plots(df):
    fig, ax = plt.subplots(2, figsize=(12,6))
    plt.xlabel('Lag Order')
    ax[0] = plot_acf(df, ax=ax[0])
    ax[1] = plot_pacf(df, ax=ax[1])

get_acf_pacf_plots(series[5::6])
plt.show()

In [ ]:
series2 = series[119::120]
get_acf_pacf_plots(series2)
plt.show()

In [ ]:
series2 = series2[30::31]
get_acf_pacf_plots(series2)
plt.show()

In [ ]:
from scipy.fft import fft

fft_vals = fft(series[5::6].values)
f_per_dataset = np.arange(0, len(fft_vals))

n_samples_h = len(series[5::6])

hours_per_year = 24*365.2524
years_per_dataset = n_samples_h/(hours_per_year)

f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft_vals))
plt.xscale('log')
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')
plt.show()


In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_dickey_fuller_stationarity(df):
    dftest = adfuller(df, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic',
                                             'p-value',
                                             'Number of Lags Used',
                                             'Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    if dfoutput['Critical Value (1%)'] <  dfoutput['Test Statistic']:
        print('Series is not stationary with 99% confidence. ')
    elif dfoutput['Critical Value (5%)'] < dfoutput['Test Statistic']:
        print('Series is not stationary with 95% confidence. ')
    elif dfoutput['Critical Value (10%)'] < dfoutput['Test Statistic']:
        print('Series is not stationary with 90% confidence. ')
    else:
        print('Series is possibly stationary. ')
    return dfoutput

In [ ]:
from statsmodels.tsa.stattools import kpss

def test_kpss(df):
    dftest = kpss(df)
    dfoutput = pd.Series(dftest[0:3], index=['Test Statistic',
                                             'p-value',
                                             'Number of Lags Used'])
    for key,value in dftest[3].items():
        dfoutput['Critical Value (%s)'%key] = value
    if abs(dfoutput['Critical Value (1%)']) < abs(dfoutput['Test Statistic']):
        print('Series is not stationary with 99% confidence. ')
    elif abs(dfoutput['Critical Value (5%)']) < abs(dfoutput['Test Statistic']):
        print('Series is not stationary with 95% confidence. ')
    elif abs(dfoutput['Critical Value (10%)']) < abs(dfoutput['Test Statistic']):
        print('Series is not stationary with 90% confidence. ')
    else:
        print('Series is possibly stationary. ')
    return dfoutput

In [ ]:
def eliminate_trends(series):
    roll = series.rolling(4).mean()
    avg_diff = (series - roll)/roll
    avg_diff.dropna(inplace=True)
    return avg_diff

diff = eliminate_trends(series[5::6])
print('-------- ADF Test ---------')
out = test_dickey_fuller_stationarity(diff)
print(out)
print('-------- KPSS Test ---------')
out = test_kpss(diff)
print(out)

In [ ]:
def eliminate_linear_trend(series):
    x = np.arange(len(series))
    m, c = np.polyfit(x, series, 1)
    return (series - c) / m
    
diff = eliminate_linear_trend(series[5::6])
print('-------- ADF Test ---------')
out = test_dickey_fuller_stationarity(diff)
print(out)
print('-------- KPSS Test ---------')
out = test_kpss(diff)
out = print(out)

In [ ]:
def difference(series, lag=1):
    differenced = []
    for x in range(lag, len(series)):
        differenced.append(series[x] - series[x - lag])
    return pd.Series(differenced)

diff = difference(series[5::6])
print('-------- ADF Test ---------')
out = test_dickey_fuller_stationarity(diff)
print(out)
print('-------- KPSS Test ---------')
out = test_kpss(diff)
print(out)